# Problem raspoređivanja

U susedstvu se otvara nova prodavnica koja će biti otvorena 24 sata dnevno, 7 dana u nedelji. Da bi se pokrio obim posla, radnici će raditi u tri smene u trajanju po osam sati. Jutarnja smena je od 6:00 do 14:00, poslepodnevna smena je od 14:00 do 22:00, a noćna smena je od 22:00 do 6:00 sledećeg dana.

Za vreme noćne smene potreban je jedan radnik, dok su za rad u dnevnim smenama potrebna po dva radnika. Izuzetak je nedelja, kada je i u dnevnim smenama potreban po jedan radni. Svaki radnik ima opterećenje od najviše 40 sati nedeljno i treba da se odmori barem 12 sati između dveju smena. Radnici koji ne rade nedeljom preferiraju da imaju i slobodnu subotu. 

Menadžeru koji treba da napravi raspored je na raspolaganju 10 radnika. Potrebno je da napravi optimalni raspored sa što manjim brojem radnika kako bi preostali radnici mogli da pomognu u drugim prodavnicama.

In [1]:
from pyomo.environ import *

Prvo ćemo izdvojiti podatke koji se tiču same formulacije problema. Lista `days` će sadržati imena radnih dana, lista `shifts` će sadržati imena smena, a rečnik `days_shift` njihovu kombinaciju. 

In [2]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
shifts = ['morning', 'evening', 'night']

# ime dana je kljuc recnika, a lista smena vrednost
days_shifts = {day: shifts for day in days} 

In [3]:
days_shifts

{'Mon': ['morning', 'evening', 'night'],
 'Tue': ['morning', 'evening', 'night'],
 'Wed': ['morning', 'evening', 'night'],
 'Thu': ['morning', 'evening', 'night'],
 'Fri': ['morning', 'evening', 'night'],
 'Sat': ['morning', 'evening', 'night'],
 'Sun': ['morning', 'evening', 'night']}

Informacije o raspoloživim radnicima ćemo čuvati na nivou liste `workers`.

In [4]:
number_of_workers = 10
workers = ['W' + str(i+1) for i in range(0, number_of_workers)]

In [5]:
workers

['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10']

Dalje ćemo kreirati model. 

In [6]:
model = ConcreteModel()

Promenljive `works` će biti indeksirane identifikatorima radnika, imenima dana i imenima smena i imaće binarnu vrednost. Vrednost `0` će označavati da izdvojeni radnik nije raspoređen za rad u smeni u određenom danu (to će biti i početna vrednost promenljivih), dok će vrednost `1` označavati da je radnik raspoređen za rad.

In [7]:
model.works = Var(((worker, day, shift) for worker in workers for day in days for shift in days_shifts[day]), within=Binary, initialize=0)

In [8]:
model.works['W1', 'Mon', 'morning'].value

0

Promenljive `needed` će označavati da li je radnik neophodan za rad u smeni. Biće indeksirane identifikatorima radnika, takođe binarne vrednosti sa nulom kao početnom vrednošću.

In [9]:
model.needed = Var(workers, within=Binary, initialize=0)

In [10]:
model.needed['W1'].value

0

Promenljive `no_pref` će pratiti aktivnosti radnika u toku vikenda. Takođe će biti binarnih vrednosti, sa vrednošću `1` ukoliko radnik radi u nedelju, a ne u subotu. Probaćemo da izbegnemo ovakve scenarije.

In [11]:
model.no_pref = Var(workers, within=Binary, initialize=0)

In [12]:
model.no_pref['W1'].value

0

Ciljna funkcija će pratiti broj potrebnih radnika, kao i raspored vikendom. Faktor koji se odnosi na broj potrebnih radnika je pomnožen konstantom koja treba da da veću težinu ovom uslovu optimizacije. Funkciju cilja je potrebno minimizovati.

In [13]:
def obj_rule(model):
    # kako je suma (model.no_pref) sa najvecom vrednoscu len(workers), len(workers) + 1 je validna tezina
    c = len(workers) + 1 
    return sum(model.no_pref[worker] for worker in workers) + c*sum(model.needed[worker] for worker in workers)

model.obj = Objective(rule=obj_rule, sense=minimize)

Zatim ćemo kreirati listu ograničenja. Za njeno kreiranje iskoristićemo funkciju `ConstraintList`, a za dodavanje pojedinačnih ograničenja funkciju `add`.

In [14]:
model.constraints = ConstraintList()

Prva grupa ogrančenja se odnosi na pokrivenost svih smena.

In [15]:
for day in days:
    for shift in days_shifts[day]:
        # ako je u pitanju radni dan ili subota i prepodnevna ili poslepodnevna smena
        # broj radnika u smeni treba da bude 2
        if day in days[:-1] and shift in ['morning', 'evening']:
            model.constraints.add(
                2 == sum(model.works[worker, day, shift] for worker in workers)
            )
        else:
            # nedeljom i u nocnoj smeni broj radnika treba da bude 1
            model.constraints.add(
                1 == sum(model.works[worker, day, shift] for worker in workers)
            )

Druga grupa ograničenja se odnosi na radno vreme radnika - radnik ne radi više od 40 sati nedeljno.

In [16]:
for worker in workers:
    model.constraints.add(
        40 >= sum(8 * model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    )

Treća grupa ograničenje se tiče razmaka između dveju smena koji treba da bude barem 12 sati.

In [17]:
for worker in workers:
    for j in range(len(days)):
        
        # radnik ne moze da radi u dvema smenama u istom danu
        model.constraints.add(
            1 >= sum(model.works[worker, days[j], shift] for shift in days_shifts[days[j]])
        )
        # ako radnik radi poslepodnevnu ili nocnu smenu, ne moze da radi u prepodnevnoj smeni sledeceg dana
        # praticemo i uslov da posle nedelje dolazi ponedeljak preko izraza (j+1)%7 
        model.constraints.add(
            1 >= sum(model.works[worker, days[j], shift] for shift in ['evening', 'night']) +
            model.works[worker, days[(j + 1) % 7], 'morning']
        )
        # ako radnik radi nocnu smenu, ne moze da radi prepodnevnu ili poslepodnevnu smenu sledeceg dana
        model.constraints.add(
            1 >= model.works[worker, days[j], 'night'] +
            sum(model.works[worker, days[(j + 1) % 7], shift] for shift in ['morning', 'evening'])
        )

Četvrta grupa ograničenja se tiče potrebnosti radnika. 

In [18]:
# jedan radnik radi najvise 40 sati tj. najvise 5 smena
for worker in workers:
    model.constraints.add(
        # ako je vrednost model.works[worker, ·, ·] razlicita od nule, model.needed[worker] mora biti jedan
        # u usprotnom vrednost traba da bude nula zbog minimizacije funkcije cilja
        5 * model.needed[worker] >= sum(model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    ) 

Poslednja grupa ograničenja se tiče rada vikendom.

In [19]:
# ako radnik ne radi u nedelju, pozeljno je da ne radi ni u subotu
for worker in workers:
    # ako radnik ne radi u nedelju, ali radi u subotu, vrednost no_pref promenljive mora da bude 1
    # ako radnik radi u nedelju, vrednost no_pref promenljive ce biti 0 (zbog minimizacije funkcije cilja)
    model.constraints.add(
        model.no_pref[worker] >= sum(model.works[worker, 'Sat', shift] for shift in days_shifts['Sat'])
        - sum(model.works[worker, 'Sun', shift] for shift in days_shifts['Sun'])
    )

Ukupan broj ograničenja koji je postavljen na nivou modela je: 

In [20]:
len(model.constraints)

261

Sam model je, takođe, vrlo kompleksan: 

In [21]:
# model.pprint()

Kako problem koji rešavamo predstavlja varijantu mešovitog celobrojnog programiranja, dovoljno je upotrebiti rešavač koji podršava rad sa ovom grupom problema. Mi ćemo iskoristiti GLPK rešavač. Rešavanje problema traje par sati. 

In [ ]:
opt = SolverFactory('glpk')  
results = opt.solve(model)  

Sledeće funkcije će nam pomoći da izdvojimo informacije koje je generisao rešavač. 

Funkcije `get_workers_needed` izdvaja listu potrebnih radnika. 

In [ ]:
def get_workers_needed(needed):
    workers_needed = []
    for worker in workers:
        if needed[worker].value == 1:
            workers_needed.append(worker)
    return workers_needed

Funkcija `get_work_table` izdvaja raspored rada radnika. 

In [ ]:
def get_work_table(works):
    # raspored rada radnika ce biti u formi recnika
    week_table = {day: {shift: [] for shift in days_shifts[day]} for day in days}
    for worker in workers:
        for day in days:
            for shift in days_shifts[day]:
                    if works[worker, day, shift].value == 1:
                        week_table[day][shift].append(worker)
    return week_table

Funkcija `get_no_preference` izdvaja listu radnika za koje nije ispunjen uslov rada vikendom.

In [ ]:
def get_no_preference(no_pref):
    return [worker for worker in workers if no_pref[worker].value == 1]

Ostaje još da izdvojimo potrebne informacije i prikažemo ih.

In [ ]:
workers_needed = get_workers_needed(model.needed)  

In [ ]:
print('Potrebni radnici: ', workers_needed)

In [ ]:
week_table = get_work_table(model.works)

In [ ]:
print('Plan rada radnika: ', week_table)

Rešenje koje se generiše:

In [ ]:
{
  "Mon": {
    "morning": [
      "W5",
      "W8"
    ],
    "evening": [
      "W6",
      "W7"
    ],
    "night": [
      "W9"
    ]
  },
  "Tue": {
    "morning": [
      "W5",
      "W8"
    ],
    "evening": [
      "W1",
      "W7"
    ],
    "night": [
      "W9"
    ]
  },
  "Wed": {
    "morning": [
      "W5",
      "W10"
    ],
    "evening": [
      "W6",
      "W7"
    ],
    "night": [
      "W9"
    ]
  },
  "Thu": {
    "morning": [
      "W1",
      "W5"
    ],
    "evening": [
      "W7",
      "W8"
    ],
    "night": [
      "W6"
    ]
  },
  "Fri": {
    "morning": [
      "W1",
      "W5"
    ],
    "evening": [
      "W7",
      "W10"
    ],
    "night": [
      "W6"
    ]
  },
  "Sat": {
    "morning": [
      "W8",
      "W9"
    ],
    "evening": [
      "W1",
      "W10"
    ],
    "night": [
      "W6"
    ]
  },
  "Sun": {
    "morning": [
      "W8"
    ],
    "evening": [
      "W10"
    ],
    "night": [
      "W1"
    ]
  }
}


In [ ]:
workers_no_pref = get_no_preference(model.no_pref)

In [ ]:
print('Radnici koji ce imati problematican vikend: ', workers_no_pref)